In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.0 MB/s 
     |████████████████████████████████| 7.6 MB 55.5 MB/s 
     |████████████████████████████████| 182 kB 61.4 MB/s 


In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, RobertaTokenizer, RobertaForSequenceClassification, GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import pandas as pd
import pickle
import os
import argparse
import numpy as np
from tqdm import tqdm
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
sw = stopwords.words('english') 

from collections import Counter
c = Counter()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

tokenizer_roberta = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
model_roberta = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

tokenizer_gpt = GPT2Tokenizer.from_pretrained('gpt2')
with torch.no_grad():
    model_gpt = GPT2LMHeadModel.from_pretrained('gpt2')
    model_gpt.eval()

model_random = torch.load('/content/drive/MyDrive/ANLP-project/bert_detox_rand_ft_10epochs.pth',map_location=torch.device('cpu'))
model_targeted = torch.load('/content/drive/MyDrive/ANLP-project/bert_detox_ft_5epochs.pth',map_location=torch.device('cpu'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another archite

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Identifying the toxic word

### Bag of words

In [ ]:
toxic_word_path='../data/vocab/negative_words.txt'

with open(toxic_word_path, 'r') as f:
    toxic_word_list = f.read().splitlines()
toxic_word_list = [line.strip() for line in toxic_word_list]

def bag_of_words(sentence, list=toxic_word_list):

    sentence_list = sentence.split()

    for word in sentence_list:
        if word in list:
            sentence = sentence.replace(word, "[MASK]")


    return sentence


### Logistic Regression

In [ ]:
!pip install scikit-learn
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

In [ ]:
X_train = corpus_tox + corpus_norm
y_train = [1] * len(corpus_tox) + [0] * len(corpus_norm)
pipe.fit(X_train, y_train)

In [ ]:
coefs = pipe[1].coef_[0]
coefs.shape

In [ ]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [ ]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

### Roberta

In [ ]:
def mask(sentence):
  # sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
  
  sentence = ' ' + sentence + ' '
  sentence = sentence.lower()
  # adding spaces around punctuation
  sentence = re.sub(r'([.,!?()])', r' \1 ', sentence)

  sentence = sentence.split(' ')

  # true_sent = []
  # for i in sentence:
  #   true_sent.append(i)
  # true_sent = sentence
  true_sent = sentence.copy()

  # print(true_sent)
  masked_sentence = ''
  min_tox = 1
  while True:
    for i in range(len(sentence)):

      # print('1:',i,sentence)
      sentence[i] = '[MASK]'
      # print('2:',i,sentence)
      sentence = ' '.join(sentence)
      batch = tokenizer_roberta.encode(sentence, return_tensors='pt')
      output = torch.nn.functional.softmax(model_roberta(batch).logits, dim = -1)
      toxic_score = float(output[0][1])
      if min_tox > toxic_score:
        min_tox = toxic_score
        masked_sentence = sentence

      # print('nvwubrb',true_sent)
      sentence = true_sent.copy()

    
    # if min_tox < 0.25:
    #   break
    break
    
    true_sent = masked_sentence
    true_sent = true_sent.split(' ')
  #join the sentence
  
  
  return masked_sentence

In [ ]:
mask("I fucking love you.")

' i [MASK] love you .  '

## Use the fine-tuned BERT to predict alternatives

### BERT Model: Random mask

In [ ]:
#loading the model
def predictor_random(sentence, N):
    inputs = tokenizer.batch_encode_plus([sentence], return_tensors='pt')
    inputs["token_type_ids"] = torch.zeros(len(inputs.input_ids)).to(torch.int64) # non-toxic

    with torch.no_grad():
        logits = model_random(**inputs).logits
    
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    values, indices = torch.topk(logits[0, mask_token_index, :], N)

    sentences_list = []
    
    for i in range(N):
        inputs.input_ids[0][mask_token_index.item()] = indices[0][i]
        sentences_list.append(tokenizer.decode(inputs.input_ids[0]))
        # sentences_list.append()
    
    # sentence = sentence[:sentence.find('[MASK]')] + " [MASK] " + sentence[sentence.find('[MASK]'):]

    # inputs = tokenizer.batch_encode_plus([sentence], return_tensors='pt')
    # inputs["token_type_ids"] = torch.zeros(len(inputs.input_ids)).to(torch.int64) # non-toxic

    # with torch.no_grad():
    #     logits = model_random(**inputs).logits
    
    # mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    # values, indices = torch.topk(logits[0, mask_token_index, :], N)

    # # print(indices)

    # for i in range(N):  
    #     inputs.input_ids[0][mask_token_index] = torch.tensor([indices[0][i],indices[1][i]])
    #     sentences_list.append(tokenizer.decode(inputs.input_ids[0]))
    #     # sentences_list.append()



    return sentences_list






### BERT Model: Targetted mask

In [ ]:
def predictor_targeted(sentence, N):
    inputs = tokenizer.batch_encode_plus([sentence], return_tensors='pt')
    inputs["token_type_ids"] = torch.zeros(len(inputs.input_ids)).to(torch.int64) # non-toxic

    with torch.no_grad():
        logits = model_targeted(**inputs).logits
    
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    values, indices = torch.topk(logits[0, mask_token_index, :], N)

    sentences_list = []
    # replace every mask token by the top N tokens
    # and add the sentences to the list

    
    for i in range(N):
        inputs.input_ids[0][mask_token_index.item()] = indices[0][i]
        sentences_list.append(tokenizer.decode(inputs.input_ids[0]))
        # sentences_list.append()

    
    # sentence = sentence[:sentence.find('[MASK]')] + " [MASK] " + sentence[sentence.find('[MASK]'):]

    # inputs = tokenizer.batch_encode_plus([sentence], return_tensors='pt')
    # inputs["token_type_ids"] = torch.zeros(len(inputs.input_ids)).to(torch.int64) # non-toxic

    # with torch.no_grad():
    #     logits = model_random(**inputs).logits
    
    # mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    # values, indices = torch.topk(logits[0, mask_token_index, :], N)

    # # print(indices)

    # for i in range(N):  
    #     inputs.input_ids[0][mask_token_index] = torch.tensor([indices[0][i],indices[1][i]])
    #     sentences_list.append(tokenizer.decode(inputs.input_ids[0]))
    #     # sentences_list.append()

    return sentences_list

## Ranking based on Metrics

In [ ]:

def ranker( possible_sentences,original_sentence):
  max_score = -1

  detoxed_sentence = ''
  max_sim = 0
  max_tox = 0
  max_flu = 0

  for sentence in possible_sentences:

    sim = similarity_scorer(sentence, original_sentence)
    tox = 1/toxicity_scorer(sentence)
    flu = 1/fluency_scorer(sentence)
    
    # sent_score = similarity_scorer(sentence, original_sentence)/(toxicity_scorer(sentence)*fluency_scorer(sentence))
    sent_score = sim*tox*flu

    if sent_score > max_score:
      detoxed_sentence = sentence
      max_sim = sim
      max_tox = tox
      max_flu = flu

  print("Similarity: ", max_sim)
  print("Toxicity: ", max_tox)
  print("Fluency: ", max_flu)    
  
    
  return sentence

In [ ]:
sentence = "  the entire article was shit . "
print(ranker( predictor_targeted(mask(sentence),5), sentence))

Similarity:  0.628970902033151
Toxicity:  26007.29425990197
Fluency:  0.009653532798826303
[CLS] the entire article was disputed. [SEP]


In [ ]:
print(ranker( predictor_random(mask(sentence),5), sentence))


Similarity:  0.628970902033151
Toxicity:  24755.113340881442
Fluency:  0.010370140401029609
[CLS] the entire article was reported. [SEP]


In [ ]:

def Union(lst1, lst2):
    final_list = lst1 + lst2
    return final_list


    
def similarity_scorer(predicted_sentence, original_sentence):
    #cosine similarity
  l1=[]
  l2=[]

  list1 = word_tokenize(predicted_sentence)
  list2 = word_tokenize(original_sentence)

  list1 = [w for w in list1 if w not in sw]
  list2 = [w for w in list2 if w not in sw]

  rvector = Union(list1,list2)

  for w in rvector:
    if w in list1: l1.append(1)
    else: l1.append(0)
    if w in list2: l2.append(1)
    else: l2.append(0)

  c = 0

  for i in range(len(rvector)):
    c+=l1[i]*l2[i]
  
  cosine = c / float((sum(l1)*sum(l2))**0.5)

  return cosine
  

In [ ]:
def toxicity_scorer(sentence):
    #toxicity score
  batch = tokenizer_roberta.encode(sentence, return_tensors='pt')
  output = torch.nn.functional.softmax(model_roberta(batch).logits, dim = -1)
  toxic_score = float(output[0][1])
  return toxic_score

In [ ]:
def fluency_scorer(sentence):
    tokenize_input = tokenizer_gpt.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    loss=model_gpt(tensor_input, labels=tensor_input)[0]
    return np.exp(loss.detach().numpy())
    

In [ ]:
sentence = " i really love you . "

print(similarity_scorer(sentence," i fucking love you . "))
print(toxicity_scorer(sentence))
print(fluency_scorer(sentence))

0.8
4.7692290536360815e-05
171.80545


## Testing

In [ ]:
# testing the model on jigsaw database

import pickle
test = pickle.load('../data/test.csv', 'rb')

score_random = 0
score_targeted = 0

for i in test:
    score_random += (ranker( predictor_random(mask(sentence),5), sentence), sentence)
    score_targeted += (ranker( predictor_targeted(mask(sentence),5), sentence), sentence)

print(score_random/len(test))
print(score_targeted/len(test))